In [1]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm

In [6]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware

# # keep only values near the end of 2021
# hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
# hardware_instances = hardware_instances[hardware_instances["date"] < "2020-11-01"]

# unique_values = hardware_instances["hardware"].value_counts()
# pairs = []
# for i in range(0, len(unique_values)):
#     # print(unique_values.index[i], unique_values[i])
#     pairs.append((unique_values.index[i], unique_values[i]))

# pairs = sorted(pairs, key=lambda x: x[0], reverse=False)
# for pair in pairs:
#     # print(pair)
#     print(pair[0])

pd.set_option('display.max_rows', None)

# keep only values near the end of 2021
hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
hardware_instances = hardware_instances[hardware_instances["date"] < "2020-12-01"]
hardware_instances[["date", "hardware"]].to_csv("hardware_instances_2020_all.csv", index=False)

from hardware_mapping import map_hardware_to_table
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))
hardware_instances

,date,hardware,hardware_mapped
21328,2020-07-13 00:55:35,m20s,microbt whatsminer m20s
21329,2020-07-13 00:55:35,s9,antminer s9
21330,2020-07-14 01:28:22,avalonminer 821 11.0 th/s 1200w bitcoin (sha-2...,canaan avalonminer 821
21331,2020-07-14 01:28:22,microsoft word,unknown
21332,2020-07-30 20:10:22,isl8225m eval board regulator,unknown
21333,2020-08-04 22:15:17,futurebit apollo scrypt miner,unknown
21334,2020-08-04 22:15:17,apollo miner,unknown
21335,2020-08-04 22:15:17,pc,unknown
21336,2020-08-04 22:15:17,usb hub,unknown
21337,2020-08-04 22:15:17,miner,unknown


In [7]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances[hardware_instances["date"] > "2018-01-01"]
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

# find most common unique values that were mapped to "unknown"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "unknown"]["hardware"].value_counts()
unique_values[:100]

hardware
psu                                     36
hashboard                               16
fan                                     12
fans                                    11
hashboards                               9
asics                                    9
miner                                    8
avalon                                   7
psus                                     7
sd card                                  7
l3+                                      7
heatsinks                                6
heatsink                                 5
raspberry pi                             5
power supply                             4
hash board                               4
s15                                      4
temp sensor                              4
pi4                                      3
hash boards                              3
r606                                     3
apw3++                                   3
cgminer                                  3
mi

In [4]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(lambda x: str(x).strip()).apply(map_hardware_to_table))

# print things that are "not found"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "not found"]["hardware"].value_counts()
unique_values[:10]

hardware
controller           46
control board        40
computer             39
controller board     35
computers            13
control unit         10
controllers           7
controller boards     5
controller card       4
connectors            4
Name: count, dtype: int64